# Imports globales y funciones de utilidad

In [1]:
import time
import torch
import itertools
import torchvision
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data.dataloader import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
def get_dataloaders(train_transf, test_transf, batch_size):
    train_dataset = CIFAR10("data", train=True, download=True, transform=train_transf)
    test_dataset = CIFAR10("data", train=False, download=True, transform=test_transf)

    train_size = int(0.8 * len(train_dataset))
    valid_size = len(train_dataset) - train_size
    train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
    valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

    return train_loader, valid_loader, test_loader

# Image Augmentation

La primera parte de este laboratorio consiste en expandir un poco el modelo de LeNet para tener más parámetros y luego explorar algunas técnicas de Image Augmentation (https://pytorch.org/vision/stable/transforms.html).

El modelo a implementar es el siguiente:


![Image](https://i.ibb.co/WxGgbmL/Capture.png)


In [3]:
class CustomCNN(nn.Module):
  def __init__(self, in_channels, number_classes):
    # in_channels: int, cantidad de canales de la imagen original
    super(CustomCNN, self).__init__()
    # Su implementacion
    self.conv1 = nn.Conv2d(in_channels, out_channels = 32, kernel_size = 3, padding = 1)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1)
    self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)
    self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1)
    self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 120, kernel_size = 3, padding = 1)

    self.linear1 = nn.Linear(in_features = 120*4*4, out_features = 512)
    self.linear2 = nn.Linear(in_features = 512, out_features = number_classes)

    self.max_pooling = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    # Su implementacion
    out = F.relu(self.conv1(x))
    out = F.relu(self.conv2(out))
    out = self.max_pooling(out)

    out = F.relu(self.conv3(out))
    out = F.relu(self.conv4(out))
    out = self.max_pooling(out)

    out = F.relu(self.conv5(out))
    out = self.max_pooling(out)

    out = self.dropout(out.flatten(1))

    out = F.relu(self.linear1(out))
    out = self.dropout(out)
    out = self.linear2(out)
    return out

## Funciones genericas para entrenar nuestros modelos

Vamos a utilizar las mismas funciones que implementamos en los laboratorios anteriores para entrenar y testear nuestros modelos.

In [4]:
def train_epoch(training_model, loader, criterion, optim):
    training_model.train()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    for images, labels in loader:
      all_labels.extend(labels.numpy())

      optim.zero_grad()

      predictions = training_model(images.to(device))
      all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

      loss = criterion(predictions, labels.to(device))

      loss.backward()
      optim.step()

      epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def validation_epoch(val_model, loader, criterion):
    val_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
      for images, labels in loader:
        all_labels.extend(labels.numpy())

        predictions = val_model(images.to(device))
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels.to(device))

        epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def train_model(model, train_loader, test_loader, criterion, optim, number_epochs):
  train_history = []
  test_history = []
  accuracy_history = []

  for epoch in range(number_epochs):
      start_time = time.time()

      train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, train_acc, time.time() - start_time))

      start_time = time.time()
      test_loss, acc = validation_epoch(model, test_loader, criterion)
      test_history.append(test_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, test_loss, acc, time.time() - start_time))

## Entrenando modelos

Comenzamos definiendo una seccion de código con valores por defecto de hiperparámetros que vamos a utilizar y luego entrenamos un modelo de la CNN definida anteriormente sin usar augmentation en los datos y otro haciendo uso del mismo.

https://pytorch.org/vision/stable/transforms.html

In [5]:
# Global models config

BATCH_SIZE = 10
LR = 0.001
NUMBER_EPOCHS = 15
criterion = nn.CrossEntropyLoss().to(device)

In [6]:
# Fijamos las semillas siempre para poder comparar.

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los dataloaders
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.ToTensor()
])

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

# Definimos el modelo y el optimizador
modelo_sin_aug = CustomCNN(3,10).to(device)
optimizer = torch.optim.Adam(modelo_sin_aug.parameters(), lr=LR)

# Entrenamos
train_model(modelo_sin_aug, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

100%|██████████| 170498071/170498071 [00:02<00:00, 74465370.72it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training epoch 1 | Loss 1.693512 | Accuracy 37.23% | Time 38.33 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Validation epoch 14 | Loss 0.890303 | Accuracy 69.07% | Time 5.36 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training epoch 15 | Loss 0.912930 | Accuracy 67.93% | Time 28.37 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation epoch 15 | Loss 0.867402 | Accuracy 70.23% | Time 4.37 seconds


In [7]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los datasets
test_transform = transforms.Compose([
    transforms.ToTensor()
])

# Definir transormaciones que vamos a aplicar al set de entrenamiento
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor()
])

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)


# Crear el modelo, optimizador y entrenarlo
modelo_con_aug = CustomCNN(3,10).to(device)
optimizer = torch.optim.Adam(modelo_con_aug.parameters(), lr=LR)

# Entrenamos
train_model(modelo_con_aug, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training epoch 1 | Loss 2.302427 | Accuracy 9.71% | Time 29.74 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation epoch 1 | Loss 2.302992 | Accuracy 9.73% | Time 4.73 seconds


...

Validation epoch 14 | Loss 2.302940 | Accuracy 9.67% | Time 4.75 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training epoch 15 | Loss 2.302920 | Accuracy 9.61% | Time 31.89 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation epoch 15 | Loss 2.302934 | Accuracy 9.98% | Time 5.68 seconds


## Evaluando los modelos en los datos de test

Vamos a comenzar evaluando en los datos de test normales y luego vamos a aplicar distintas transformaciones (para simular entornos más reales de datos) y vamos a ver la performance y robustez de los modelos que entrenamos anteriormente.

Primero agregar horizontal flip y luego vertical flip, qué pasa con los modelos?

In [8]:
test_transform = transforms.Compose([
  transforms.ToTensor()
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

# Testear usando las funciones definidas anteriormente
loss, acc = validation_epoch(modelo_con_aug, test_loader, criterion)
print('Test Loss:', loss, '- Test Acc:',acc)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test Loss: 2.3027040934562684 - Test Acc: 10.0


In [9]:
test_transform = transforms.Compose([
  # Flip Horizontal y volver a testear
  transforms.RandomHorizontalFlip(p=1),
  transforms.ToTensor()
])


_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

# Testear usando las funciones definidas anteriormente
loss, acc = validation_epoch(modelo_con_aug, test_loader, criterion)
print('Test Loss:', loss, '- Test Acc:',acc)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test Loss: 2.3027040960788727 - Test Acc: 10.0


In [10]:
test_transform = transforms.Compose([
  transforms.RandomVerticalFlip(p=1),
  transforms.ToTensor()
])


_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

# Testear usando las funciones definidas anteriormente
loss, acc = validation_epoch(modelo_con_aug, test_loader, criterion)
print('Test Loss:', loss, '- Test Acc:',acc)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test Loss: 2.3027040977478026 - Test Acc: 10.0


# DenseNet


![Image](https://miro.medium.com/max/5164/1*_Y7-f9GpV7F93siM1js0cg.jpeg)

Link al paper original: [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

Algunas consideraciones del paper a tener en cuenta:

1. Batch normalization en los inputs de los bloques densos y las capas de transición.
2. ReLU en todos lados como funcion de activación.
3. El MLP al final de la red cuenta con una capa oculta de 512 neuronas
4. Las activaciones luego del tercer bloque denso tienen tamaño 4*4 (ejercicio, calcular a mano!)


Implementamos DenseNet para resolver el problema de CIFAR10


In [11]:
import math
def compute_same_padding(input_shape, strides, kernel_size):
    padding_total = (input_shape*strides-input_shape-strides+kernel_size)/2
    return math.ceil(padding_total/2)

In [12]:
class CompositeFunction(nn.Module):
    #BatchNorm + Relu + Conv
    def __init__(self, in_channels, out_channels, kernel_size, input_shape, strides = 1):
        super(CompositeFunction, self).__init__()
        self.bn = nn.BatchNorm2d(num_features = in_channels)
        P = compute_same_padding(input_shape, strides, kernel_size)
        self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, padding = P)

    def forward(self, x):
        out = self.bn(x)
        out = self.conv(F.relu(out))
        return out

In [13]:
class DenseBlock(nn.Module):
  #out_channels = in_channels + k * reps
  #bottleneck = 4*k
  def __init__(self, in_channels, reps, k, input_shape):
    super(DenseBlock, self).__init__()
    # Su implementacion
    self.reps = reps
    self.convs = []
    for i in range(reps):
        self.convs.append(CompositeFunction(in_channels+k*i, 4*k, 1, input_shape))
        self.convs.append(CompositeFunction(4*k, k, 3, input_shape))

    self.convs = nn.ModuleList(self.convs)

  def forward(self, x):
    for i in range(self.reps):
        x1 = self.convs[2*i](x)
        x2 = self.convs[2*i+1](x1)
        x = torch.cat([x,x2],1)
    return x

In [14]:
class TransitionLayer(nn.Module):
  def __init__(self, in_channels, out_channels, input_shape):
    super(TransitionLayer, self).__init__()
    self.comp = CompositeFunction(in_channels, out_channels, 1, input_shape)
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self, x):
    out = self.comp(x)
    out = self.pool(out)
    return out

In [15]:
class DenseNet(nn.Module):
    #DenseNet 121
    def __init__(self, n_classes, input_shape, k):
        super(DenseNet, self).__init__()
        self.input_conv = CompositeFunction(in_channels = 3, out_channels = 2*k ,kernel_size = 7, input_shape = input_shape, strides = 2)
        self.input_max_pooling = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.dense_block1 = DenseBlock(2*k, 6, k, int(input_shape/2))
        self.transitionLayer1 = TransitionLayer(in_channels=(6+2)*k, out_channels=4*k,  input_shape = int(input_shape/2))

        self.dense_block2 = DenseBlock(4*k, 12, k, int(input_shape/4))
        self.transitionLayer2 = TransitionLayer(in_channels=(12+4)*k, out_channels=8*k, input_shape =int(input_shape/4))

        self.dense_block3 = DenseBlock(8*k, 24, k, int(input_shape/8))
        self.transitionLayer3 = TransitionLayer(in_channels=(24+8)*k, out_channels=16*k, input_shape =int(input_shape/8))

        self.dense_block4 = DenseBlock(16*k, 16, k, input_shape =int(input_shape/16))
        last_number_of_filters = 16*k+16*k
        # Classifier
        h = int(input_shape/16)
        w = h
        self.fully_connected_1 = nn.Linear(last_number_of_filters*w*h, 512)
        self.output = nn.Linear(512, n_classes)


    def forward(self, x):
        out = self.input_conv(x)
        out = self.input_max_pooling(out)
        out = self.dense_block1(out)
        out = self.transitionLayer1(out)
        out = self.dense_block2(out)
        out = self.transitionLayer2(out)
        out = self.dense_block3(out)
        out = self.transitionLayer3(out)
        out = self.dense_block4(out)
        out = out.flatten(1)
        out = F.relu(self.fully_connected_1(out))
        out = self.output(out)

        return out

In [16]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los datasets
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

densenet = DenseNet(10, 32, 4).to(device)
optimizer = torch.optim.Adam(densenet.parameters(), lr=LR)

BATCH_SIZE = 10
NUMBER_EPOCHS = 10

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

train_model(densenet, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training epoch 1 | Loss 1.645219 | Accuracy 39.92% | Time 290.13 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation epoch 1 | Loss 1.363611 | Accuracy 50.90% | Time 26.50 seconds


...

Validation epoch 9 | Loss 0.813575 | Accuracy 71.93% | Time 23.63 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training epoch 10 | Loss 0.719635 | Accuracy 74.97% | Time 277.10 seconds


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation epoch 10 | Loss 0.786275 | Accuracy 72.53% | Time 24.81 seconds


In [17]:
test_transform = transforms.Compose([
  transforms.ToTensor()
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

test_loss, accuracy = validation_epoch(densenet, test_loader, criterion)
print(f"DenseNet Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


DenseNet Test set: 0.776248 Loss. Accuracy 73.55%
